In [25]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

In [4]:
auth_url= client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

In [5]:
code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)


In [6]:
client = stravalib.client.Client(access_token=access_token)
athlete = client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))


athlete name Brian, athlete id 3360678.


In [7]:
import datetime
lap_start_date = datetime.datetime(2017, 9, 28)
lap_start_date

datetime.datetime(2017, 9, 28, 0, 0)

In [11]:
runs = []
for activity in client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        print(activity.name, activity.type, activity.start_date)
        runs.append(activity)

Ultimate Run 2017-09-29 17:16:08+00:00
Ultimate Run 2017-10-04 00:29:08+00:00


In [13]:
for run in runs:
    print(run)
    for lap in runs[1].laps:
        print(lap.elapsed_time)

<Activity id=1207538661 name='Ultimate' resource_state=2>
0:07:56
0:04:53
0:03:15
<Activity id=1214171210 name='Ultimate' resource_state=2>
0:07:56
0:04:53
0:03:15


In [40]:
lap

<ActivityLap id=3766238393 name='Lap 1' resource_state=2>

In [23]:

def xml_to_float(element):
    if element is not None:
        return float(element[0].firstChild.data)
    else:
        return np.nan
        
def xml_to_time(element):
    if element is not None:
        return datetime.datetime.strptime(element[0].firstChild.data, '%Y-%m-%dT%H:%M:%S.%fZ')
#         return dateutil.parser.parse(element[0].firstChild.data)
    else:
        return np.nan

In [106]:
class Dummy(object):
    pass

self = Dummy()

filename = 'activity_2040473210.tcx'   
       
    
    
doc = minidom.parse(filename)
doc.normalize()
self.tcx = doc.documentElement   

points = []

lap_starts = []
for lap in self.tcx.getElementsByTagName('Lap'):
    lap_starts.append(lap.attributes.items()[0][1])
    
lap_starts = [datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.000Z') for t in lap_starts]



#     for track in lap.getElementsByTagName('Track'):
#         for trackpoint in track.getElementsByTagName('Trackpoint'):     

#             _pos = trackpoint.getElementsByTagName('Position') or None
#             if _pos:
#                 _lat = _pos[0].getElementsByTagName('LatitudeDegrees')
#                 _lon = _pos[0].getElementsByTagName('LongitudeDegrees')
#             else:
#                 _lat = None
#                 _lon = None

#             _time = trackpoint.getElementsByTagName('Time')
#             _dist = trackpoint.getElementsByTagName('DistanceMeters') or None
#             _alt = trackpoint.getElementsByTagName('AltitudeMeters') or None

#             _hr_cont = trackpoint.getElementsByTagName('HeartRateBpm') or None
#             if _hr_cont:
#                 _hr = _hr_cont[0].getElementsByTagName('Value') or None
#             else:
#                 _hr = None

#             _cad = trackpoint.getElementsByTagName('Cadence') or None



#             _point = {'lat': xml_to_float(_lat),
#                       'lon': xml_to_float(_lon),
#                       'time': xml_to_time(_time),
#                       'alt':  xml_to_float(_alt),
#                       'hr':   xml_to_float(_hr),
#                       'cad':  xml_to_float(_cad),
#                       'dist': xml_to_float(_dist),
#                      }
#             points.append(_point)

# self.points = points
# # self.activity_type = self.tcx.getElementsByTagName('Activity')[0].attributes.getNamedItem('Sport')
# self.activity_type = 'Biking'

In [107]:
press_delta = datetime.timedelta(0,3)

presses = 1

events = []

for n in range(len(lap_starts) - 1):
    
    
    time = lap_starts[n]
    
    dt = lap_starts[n+1] - lap_starts[n]
    
    # print(presses, time, dt, dt > press_delta)
    
    
    if dt > press_delta:
        events.append([presses, time])
        presses = 1
        
    else:        
        presses += 1
        
events.append([presses, time])



In [108]:
events

[[1, datetime.datetime(2017, 10, 4, 1, 49, 57)],
 [1, datetime.datetime(2017, 10, 4, 1, 53, 57)],
 [2, datetime.datetime(2017, 10, 4, 1, 54, 28)],
 [3, datetime.datetime(2017, 10, 4, 1, 54, 48)],
 [2, datetime.datetime(2017, 10, 4, 1, 55, 37)],
 [4, datetime.datetime(2017, 10, 4, 1, 56)],
 [1, datetime.datetime(2017, 10, 4, 1, 56, 16)],
 [2, datetime.datetime(2017, 10, 4, 1, 56, 30)]]

In [109]:
event_lookup = [
    '',
    'my_point',
    'team_point',
    'opponent_point',
    'game',
]

In [114]:
games = []

base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0}
game = base_game
for event in events:
    game['end_time'] = event[1]

    event_type = event_lookup[event[0]]
    if event_type == 'game':
        games.append(game)
        game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0}
        
    elif event_type == 'my_point':
        game[event_type] += 1
        game['team_point'] += 1
    else:
        game[event_type] += 1

games.append(game)
games

[{'end_time': datetime.datetime(2017, 10, 4, 1, 56),
  'my_point': 2,
  'opponent_point': 1,
  'team_point': 4},
 {'end_time': datetime.datetime(2017, 10, 4, 1, 56, 30),
  'my_point': 1,
  'opponent_point': 0,
  'team_point': 2}]

In [115]:
import pandas as pd

In [116]:
pd.DataFrame(games)

,end_time,my_point,opponent_point,team_point
0,2017-10-04 01:56:00,2,1,4
1,2017-10-04 01:56:30,1,0,2
